# SDM1 : Montane woodcreper - Gecomputation for the Random Forest model using GRASS
One of the first action in Species Distribution Model is to record presence/absence of the species and then search for raster or vector files that store information about environment variables, that define the ecological niche. We will first concentrate on the collection of the environment variables.  

Open the bash terminal, migrate in the directory, and open the jupter lab

    cd /media/sf_LVM_shared/my_SE_data/exercise
    wget https://raw.githubusercontent.com/selvaje/SE_data/master/exercise/SDM1_MWood_gecomp4GRASS.ipynb
    jupyter lab SDM1_MWood_gecomp4GRASS.ipynb

## Crop enviromental layers

First select a dataset that we can use to define the overall extent of the specie - the study area.
We will therefore crop geodata using the species range as an extent and increase of 3 degree in N-S and W-E direction.

In [1]:
%%bash
cd /media/sf_LVM_shared/my_SE_data/exercise
# test 
# define north south weast owest extent
ulx=$(ogrinfo -al -so geodata/shp/cartodb-query.shp | grep Extent | awk '{ gsub("\\("," "); print int($2 -3)}')
uly=$(ogrinfo -al -so geodata/shp/cartodb-query.shp | grep Extent | awk '{ gsub("\\)"," "); print int($6 +3)}')
lrx=$(ogrinfo -al -so geodata/shp/cartodb-query.shp | grep Extent | awk '{ gsub("\\("," "); print int($5 +3)}')
lry=$(ogrinfo -al -so geodata/shp/cartodb-query.shp | grep Extent | awk '{ gsub("\\)"," "); print int($3 -3)}')

## Print the extent to confirm the lat long  
echo $ulx $uly $lrx $lry
## croping
gdal_translate -projwin  $ulx $uly $lrx $lry -co COMPRESS=DEFLATE -co ZLEVEL=9 \
geodata/cloud/SA_intra.tif  geodata/cloud/SA_intra_crop.tif

gdal_translate -projwin  $ulx $uly $lrx $lry -co COMPRESS=DEFLATE -co ZLEVEL=9 \
geodata/cloud/SA_meanannual.tif  geodata/cloud/SA_meanannual_crop.tif

gdal_translate -projwin  $ulx $uly $lrx $lry -co COMPRESS=DEFLATE -co ZLEVEL=9 \
geodata/dem/SA_elevation_mn_GMTED2010_mn.tif  geodata/dem/SA_elevation_mn_GMTED2010_mn_crop.tif

gdal_translate -projwin  $ulx $uly $lrx $lry -co COMPRESS=DEFLATE -co ZLEVEL=9 \
geodata/vegetation/SA_tree_mn_percentage_GFC2013.tif  geodata/vegetation/SA_tree_mn_percentage_GFC2013_crop.tif


-82 14 -59 -21
Input file size is 5880, 8400
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 5880, 8400
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 5880, 8400
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 5880, 8400
0...10...20...30...40...50...60...70...80...90...100 - done.


## Download climate layers

Dowonload climate data from https://chelsa-climate.org/ . Using the /vsicurl/ option in gdal is possible to download only the raster extend of your study area.

1) Download and process climate data (grade 20%)

Proceed with the download also for maximum temperature (tasmax), minimum temperature (tasmin), precipitation (pr), build up the loop and also calculate mean and stdev. Consider that, pkstatprofile even giving the compression flag (-co COMPRESS=DEFLATE -co ZLEVEL=9) does not compress very well so insert a gdal_translate to perform the compression (and of course remove the intermediate files).



In [9]:
%%bash
cd /media/sf_LVM_shared/my_SE_data/exercise
# download temperature tmin and tmax from https://os.zhdk.cloud.switch.ch/chelsav2/GLOBAL/climatologies/1981-2010/
for var in  pr tasmin tasmax ; do  # 
for MM in 01 02 03 04 05 06 07 08 09 10 11 12; do 
gdal_translate -a_srs EPSG:4326 -projwin -82 14 -59 -21 -co COMPRESS=DEFLATE -co ZLEVEL=9 /vsicurl/https://os.zhdk.cloud.switch.ch/chelsav2/GLOBAL/climatologies/1981-2010/${var}/CHELSA_${var}_${MM}_1981-2010_V.2.1.tif geodata/climate/CHELSA_${var}_${MM}_1981-2010_V.2.1_land_crop.tif
done
done

Input file size is 43200, 20880
0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 1: for band 1, nodata value has been clamped to 0, the original value being out of range.


Calculate annual mean and standard deviation with the monthly values.

In [10]:
%%bash
cd /media/sf_LVM_shared/my_SE_data/exercise

# calculate mean and stdev for tmax,
for var in tasmin tasmax pr ; do
gdalbuildvrt -overwrite -separate geodata/climate/CHELSA_${var}_1981-2010_V.2.1_land_crop.vrt geodata/climate/CHELSA_${var}_??_1981-2010_V.2.1_land_crop.tif

pkstatprofile -co COMPRESS=DEFLATE -co ZLEVEL=9  -nodata -32768 -f  mean -i geodata/climate/CHELSA_${var}_1981-2010_V.2.1_land_crop.vrt -o geodata/climate/CHELSA_${var}_1981-2010_V.2.1_land_crop_mean.tif 
pkstatprofile -co COMPRESS=DEFLATE -co ZLEVEL=9  -nodata -32768 -f stdev -i geodata/climate/CHELSA_${var}_1981-2010_V.2.1_land_crop.vrt -o geodata/climate/CHELSA_${var}_1981-2010_V.2.1_land_crop_stdev_tmp.tif

gdal_translate -co COMPRESS=DEFLATE -co ZLEVEL=9 geodata/climate/CHELSA_${var}_1981-2010_V.2.1_land_crop_mean_tmp.tif  geodata/climate/CHELSA_${var}_1981-2010_V.2.1_land_crop_mean.tif
gdal_translate -co COMPRESS=DEFLATE -co ZLEVEL=9 geodata/climate/CHELSA_${var}_1981-2010_V.2.1_land_crop_stdev_tmp.tif geodata/climate/CHELSA_${var}_1981-2010_V.2.1_land_crop_stdev.tif
rm -f geodata/climate/CHELSA_${var}_1981-2010_V.2.1_land_crop_*_tmp.tif
done

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 2760, 4200
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 2760, 4200
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 2760, 4200
0...10...20...30...40...50...60...70...80...90...100 - done.


ERROR 4: geodata/climate/CHELSA_tasmin_1981-2010_V.2.1_land_crop_mean_tmp.tif: No such file or directory
ERROR 4: geodata/climate/CHELSA_tasmax_1981-2010_V.2.1_land_crop_mean_tmp.tif: No such file or directory
ERROR 4: geodata/climate/CHELSA_pr_1981-2010_V.2.1_land_crop_mean_tmp.tif: No such file or directory


## Building geomorphometric layers

2) Geomorphometric layers (grade 20%)

Using the GMTED2010 DEM (geodata/dem/SA_elevation_mn_GMTED2010_mn_crop.tif) calculate "aspect" "slope" "Terrain Ruggedness Index" using gdal.
I have opt for the uploding of "Multiple Attempts", so in case you want upload another version of the homework now is possible - please do not send it by e-mail. 

In [11]:
%%bash
cd /media/sf_LVM_shared/my_SE_data/exercise

# To calculate the slope by gdaldem slope in lat long you need to setup corretly the scale flag. 
# The slope scale value is 111120 at the equator and 0 at the north/south pole. In order to get the slope correctly 
# you need first calculate the scale value in accordance to the latitudinal level of your dem 
# gdalinfo geodata/dem/SA_elevation_mn_GMTED2010_mn_crop.tif
# Center      ( -70.5000000,  -3.5000000)
# 111120 - (111120 /90 * 3.5) = 106798 

gdaldem slope  -co COMPRESS=DEFLATE -co ZLEVEL=9  -scale 106798 geodata/dem/SA_elevation_mn_GMTED2010_mn_crop.tif geodata/dem/SA_elevation_mn_GMTED2010_mn_crop_slope.tif
gdaldem aspect -co COMPRESS=DEFLATE -co ZLEVEL=9  geodata/dem/SA_elevation_mn_GMTED2010_mn_crop.tif geodata/dem/SA_elevation_mn_GMTED2010_mn_crop_aspect.tif
gdaldem TRI    -co COMPRESS=DEFLATE -co ZLEVEL=9  geodata/dem/SA_elevation_mn_GMTED2010_mn_crop.tif geodata/dem/SA_elevation_mn_GMTED2010_mn_crop_tri.tif

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


## Masking
Many of the layers that we have downloaded have the same pixel values for the sea and also for the land (e.g. percent of vegation = 0% is given to desert area and also to the see). We can use an uncilary layers (/media/sf_LVM_shared/my_SE_data/exercise/geodata/mask/msk_1km.tif) to mask all the other layers and give a common no-data value. 

In [16]:
%%bash
cd /media/sf_LVM_shared/my_SE_data/exercise

pksetmask -ot Int16 -m geodata/mask/msk_1km.tif -msknodata 0 -nodata -9999 -i geodata/cloud/SA_meanannual_crop.tif -o geodata/cloud/SA_meanannual_crop_msk.tif
pksetmask -ot Int16 -m geodata/mask/msk_1km.tif -msknodata 0 -nodata -9999 -i geodata/cloud/SA_intra_crop.tif -o geodata/cloud/SA_intra_crop_msk.tif

for var in  pr tasmin tasmax ; do  # 
for stat in stdev mean; do 
pksetmask -ot UInt16 -m geodata/mask/msk_1km.tif -msknodata 0 -nodata 65535 -i geodata/climate/CHELSA_${var}_1981-2010_V.2.1_land_crop_${stat}.tif -o geodata/climate/CHELSA_${var}_1981-2010_V.2.1_land_crop_${stat}_msk.tif
done
done

pksetmask -ot Int16 -m geodata/mask/msk_1km.tif -msknodata 0 -nodata -9999 -i geodata/dem/SA_elevation_mn_GMTED2010_mn_crop.tif -o geodata/dem/SA_elevation_mn_GMTED2010_mn_crop_msk.tif
for var in  tri slope aspect ; do
pksetmask -ot Int16 -m geodata/mask/msk_1km.tif -msknodata 0 -nodata -9999 -i geodata/dem/SA_elevation_mn_GMTED2010_mn_crop_${var}.tif -o geodata/dem/SA_elevation_mn_GMTED2010_mn_crop_${var}_msk.tif
done 

pksetmask -ot Int16 -m geodata/mask/msk_1km.tif -msknodata 0 -nodata -9999 -i geodata/vegetation/SA_tree_mn_percentage_GFC2013_crop.tif  -o geodata/vegetation/SA_tree_mn_percentage_GFC2013_crop_msk.tif

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
